In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error


In [2]:
df = pd.read_csv('../Dataset/WiproClean.csv')
df.head()

,Date,Open,High,Low,Close,Volume,Prev Close,Prev Open,PrevOPDiff,Prev High,Prev Low,Prev Volume,Time
0,2014-01-03,205.13,209.40,204.86,208.73,1107663,207.36,207.26,-0.10,208.80,204.81,915486.0,0
1,2014-01-06,209.96,211.80,208.13,209.25,1855578,208.73,205.13,-3.60,209.40,204.86,1107663.0,1
2,2014-01-07,209.25,209.51,204.38,206.33,1232292,209.25,209.96,0.71,211.80,208.13,1855578.0,2
3,2014-01-08,205.88,206.59,203.79,205.18,1077593,206.33,209.25,2.92,209.51,204.38,1232292.0,3
4,2014-01-09,205.13,207.66,201.58,202.84,1637153,205.18,205.88,0.70,206.59,203.79,1077593.0,4


In [3]:
features = df[['Open', 'Close', 'High', 'Low', 'Volume']]


In [4]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

In [5]:
dates = df['Date']

In [7]:
train_size = int(len(scaled_features) * 0.8)
train_data, val_data = scaled_features[:train_size], scaled_features[train_size:]
train_dates, val_dates = dates[:train_size], dates[train_size:]


In [8]:
def create_sequences(data, dates, sequence_length):
    X, y, seq_dates = [], [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length][1])  # Closing price is at index 1
        seq_dates.append(dates[i+sequence_length])
    return np.array(X), np.array(y), np.array(seq_dates)



In [16]:
max_sequence_length = len(val_data) - 1

# Choose the sequence length (not exceeding the maximum sequence length)
sequence_length = min(30, max_sequence_length)

# Create input sequences for training and validation
X_train, y_train, train_seq_dates = create_sequences(train_data, train_dates, sequence_length)


In [11]:
model = Sequential()
model.add(LSTM(64, input_shape=(sequence_length, 5)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')


In [12]:
model.fit(X_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
58/58 [==============================] - 2s 13ms/step - loss: 0.0027
Epoch 2/10
58/58 [==============================] - 1s 12ms/step - loss: 2.4160e-04
Epoch 3/10
58/58 [==============================] - 1s 13ms/step - loss: 2.1881e-04
Epoch 4/10
58/58 [==============================] - 1s 13ms/step - loss: 2.0200e-04
Epoch 5/10
58/58 [==============================] - 1s 14ms/step - loss: 2.0795e-04
Epoch 6/10
58/58 [==============================] - 1s 13ms/step - loss: 1.9130e-04
Epoch 7/10
58/58 [==============================] - 1s 14ms/step - loss: 1.7089e-04
Epoch 8/10
58/58 [==============================] - 1s 13ms/step - loss: 1.7954e-04
Epoch 9/10
58/58 [==============================] - 1s 12ms/step - loss: 1.9676e-04
Epoch 10/10
58/58 [==============================] - 1s 13ms/step - loss: 1.7750e-04


In [28]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense



# Extract the relevant features (Open, Close, High, Low, Volume)
features = df[['Open', 'Close', 'High', 'Low', 'Volume']]

# Normalize the features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)



# Define the sequence length (number of historical data points to consider)
sequence_length = 30

# Function to create input sequences
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length][1])  # Closing price is at index 1
    return np.array(X), np.array(y)

# Create input sequences
X, y = create_sequences(scaled_features, sequence_length)

model = Sequential()
model.add(LSTM(64, input_shape=(sequence_length, 5)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X, y, epochs=10, batch_size=32)

# Make predictions for the desired timeframe
prediction_start_date = '2023-07-20'
prediction_end_date = '2023-07-31'

# Prepare the input data for prediction
prediction_data = scaled_features[-sequence_length:]
X_pred = np.array([prediction_data])

# Generate predictions
predictions = model.predict(X_pred)

# Denormalize the predictions

predictions


Epoch 1/10
73/73 [==============================] - 3s 13ms/step - loss: 0.0084
Epoch 2/10
73/73 [==============================] - 1s 13ms/step - loss: 5.2079e-04
Epoch 3/10
73/73 [==============================] - 1s 13ms/step - loss: 4.6983e-04
Epoch 4/10
73/73 [==============================] - 1s 13ms/step - loss: 4.7986e-04
Epoch 5/10
73/73 [==============================] - 1s 13ms/step - loss: 4.3349e-04
Epoch 6/10
73/73 [==============================] - 1s 13ms/step - loss: 3.6651e-04
Epoch 7/10
73/73 [==============================] - 1s 13ms/step - loss: 3.5876e-04
Epoch 8/10
73/73 [==============================] - 1s 13ms/step - loss: 3.4273e-04
Epoch 9/10
73/73 [==============================] - 1s 13ms/step - loss: 3.2432e-04
Epoch 10/10
1/1 [==============================] - 0s 335ms/step


array([[0.44207016]], dtype=float32)